<a href="https://colab.research.google.com/github/FennicOculus/Image-translation-AutoEncoders/blob/main/Image_To_Image_Trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading Dataset

In [ ]:
! pip install -q kaggle

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

mv: cannot stat './kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle datasets download 'splcher/animefacedataset'

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip animefacedataset.zip

Archive:  animefacedataset.zip
replace images/0_2000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#Importing Libraries

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from sklearn.model_selection import train_test_split

#Encoder Part

In [ ]:
latentDim = 16
inputShape = (28, 28, 1)
chanDim = -1
depth = 1

In [ ]:
inputs = Input(shape=inputShape)
x = inputs

x = Conv2D(16, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=3)(x)

x = Conv2D(32, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=3)(x)

x = Conv2D(64, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=chanDim)(x)

volumeSize = K.int_shape(x)
x = Flatten()(x)
latent = Dense(latentDim)(x)

In [ ]:
encoder = Model(inputs, latent, name="encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 16)        160       
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 14, 14, 16)        0         
                                                                 
 batch_normalization_18 (Bat  (None, 14, 14, 16)       64        
 chNormalization)                                                
                                                                 
 conv2d_10 (Conv2D)          (None, 7, 7, 32)          4640      
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 7, 7, 32)          0         
                                                           

#Decoder part

In [ ]:
latentInputs = Input(shape=(latentDim,))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

In [ ]:
x = Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=chanDim)(x)

x = Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=chanDim)(x)

x = Conv2DTranspose(16, (3, 3), strides=2, padding="same")(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization(axis=chanDim)(x)

In [ ]:
x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
outputs = Activation("sigmoid")(x)

In [ ]:
decoder = Model(latentInputs, outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 16)]              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              17408     
                                                                 
 reshape_3 (Reshape)         (None, 4, 4, 64)          0         
                                                                 
 conv2d_transpose_12 (Conv2D  (None, 8, 8, 64)         36928     
 Transpose)                                                      
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 8, 8, 64)          0         
                                                                 
 batch_normalization_21 (Bat  (None, 8, 8, 64)         256       
 chNormalization)                                          

In [ ]:
autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                40144     
                                                                 
 decoder (Functional)        (None, 32, 32, 1)         78017     
                                                                 
Total params: 118,161
Trainable params: 117,713
Non-trainable params: 448
_________________________________________________________________


#Spliting datasets

In [ ]:
data_dir = '/content/images/'

In [ ]:
img_files = os.listdir(data_dir)

In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import cv2

In [ ]:
'''images = []
for file_name in tqdm(img_files):
    img_path = os.path.join(data_dir, file_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (80, 80))
    images.append(img)'''

'images = []\nfor file_name in tqdm(img_files):\n    img_path = os.path.join(data_dir, file_name)\n    img = cv2.imread(img_path)\n    img = cv2.resize(img, (80, 80))\n    images.append(img)'

In [ ]:
train_files, test_files = train_test_split(images, test_size=0.2, random_state=42)

In [ ]:
train_files = np.expand_dims(train_files, axis=-1)
test_files = np.expand_dims(test_files, axis=-1)
train_files = train_files.astype("float32") / 255.0
test_files = test_files.astype("float32") / 255.0

In [ ]:
train_files[0].shape

(80, 80, 3, 1)

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
((trainX, _), (testX, _)) = mnist.load_data()

In [ ]:
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

In [ ]:
EPOCHS = 25
Batch_Size = 32

In [ ]:
encoder.build(inputShape)
decoder.build(inputShape)
autoencoder.build(inputShape)

In [ ]:
autoencoder.compile(loss="mse", optimizer=Adam(learning_rate=1e-3))

In [ ]:
history = autoencoder.fit(
	trainX, trainX,
	validation_data=(testX, testX),
	epochs=EPOCHS,
	batch_size=Batch_Size)

Epoch 1/25


ValueError: ignored

In [ ]:
train_files[0]

In [ ]:
import cv2
img = cv2.imread("images/16890_2007.jpg")
img.shape